In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from scipy.stats import qmc
import regionmask

from glob import glob
import os

import dask

In [2]:
#################
## Directories ##
#################
nldas_path = '/storage/group/pches/default/public/NLDAS/'
smap_path = '/storage/group/pches/default/public/SMAP/'
project_data_path = '/storage/group/pches/default/users/dcl5300/wbm_soilM_crop_uc_lafferty-etal-2024-tbd_DATA'
log_path = '/storage/home/dcl5300/work/current_projects/wbm_soilM_crop_uc_lafferty-etal-2024-tbd/code/logs'

## Dask cluster

In [3]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    account="pches",
    # account="open",
    cores=1,
    memory="5GiB",
    walltime="02:00:00",
)

cluster.scale(jobs=30)

In [4]:
# print(cluster.job_script())

In [5]:
from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://146.186.150.11:46615,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Precalibration (central US test case)

## Setup

In [3]:
# Subset to central US
def _subset_centralUS(ds):
    # Define central US
    centralUS_states = ["Illinois", "Iowa", "Wisconsin", "Minnesota", "North Dakota", "South Dakota", "Nebraska", "Kansas", "Missouri", "Indiana", "Ohio", "Michigan", "Kentucky"]
    # Subset
    centralUS_index = regionmask.defined_regions.natural_earth_v5_0_0.us_states_50.map_keys(centralUS_states)
    centralUS_mask = regionmask.defined_regions.natural_earth_v5_0_0.us_states_50.mask(ds)
    ds_centralUS = ds.where(centralUS_mask.isin(centralUS_index), drop=True)
    # Return
    return ds_centralUS

In [4]:
##########################################
### Uncertain parameters: append range ###
##########################################
n_sample = 5_000

if True:
    ###  Define bounds
    # awCap factor range ~ 0.1 - 0.5
    awCap_factor_lower, awCap_factor_upper = 0.1, 0.5

    # wiltingp factor range ~ 0.1 - 0.5
    wiltingp_factor_lower, wiltingp_factor_upper = 0.1, 0.5

    # alpha range ~ 1 - 10
    alpha_claycoef_lower, alpha_claycoef_upper = 0.0, 10.
    alpha_sandcoef_lower, alpha_sandcoef_upper = 0.0, 10.
    alpha_siltcoef_lower, alpha_siltcoef_upper = 0.0, 10.

    # betaHBV range ~ 1 - 10
    betaHBV_claycoef_lower, betaHBV_claycoef_upper = 0.0, 10.
    betaHBV_sandcoef_lower, betaHBV_sandcoef_upper = 0.0, 10.
    betaHBV_siltcoef_lower, betaHBV_siltcoef_upper = 0.0, 10.

    # GS_start_lower, GS_start_upper = 50, 200
    # GS_length_lower, GS_length_upper = 100, 200
    # rootDepth_oGS_lower, rootDepth_oGS_upper = 100, 1200
    # rootDepth_GS_factor_lower, rootDepth_GS_factor_upper = 2., 10.
    # rootDepth_all_lower, rootDepth_all_upper = 100, 2500

    # LHC sampling
    sampler = qmc.LatinHypercube(d=8)
    sample = sampler.random(n=n_sample)

    # l_bounds = [fieldCap_claycoef_lower, fieldCap_sandcoef_lower, fieldCap_siltcoef_lower,
    #             wiltingp_claycoef_lower, wiltingp_sandcoef_lower, wiltingp_siltcoef_lower,
    #             alpha_claycoef_lower, alpha_sandcoef_lower, alpha_siltcoef_lower,
    #             betaHBV_claycoef_lower, betaHBV_sandcoef_lower, betaHBV_siltcoef_lower]
    
    # u_bounds = [fieldCap_claycoef_upper, fieldCap_sandcoef_upper, fieldCap_siltcoef_upper,
    #             wiltingp_claycoef_upper, wiltingp_sandcoef_upper, wiltingp_siltcoef_upper,
    #             alpha_claycoef_upper, alpha_sandcoef_upper, alpha_siltcoef_upper,
    #             betaHBV_claycoef_upper, betaHBV_sandcoef_upper, betaHBV_siltcoef_upper]

    l_bounds = [awCap_factor_lower,
                wiltingp_factor_lower,
                alpha_claycoef_lower, alpha_sandcoef_lower, alpha_siltcoef_lower,
                betaHBV_claycoef_lower, betaHBV_sandcoef_lower, betaHBV_siltcoef_lower]
    
    u_bounds = [awCap_factor_upper,
                wiltingp_factor_upper,
                alpha_claycoef_upper, alpha_sandcoef_upper, alpha_siltcoef_upper,
                betaHBV_claycoef_upper, betaHBV_sandcoef_upper, betaHBV_siltcoef_upper]

    sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

    # Store as csv
    df = pd.DataFrame(data = sample_scaled,
                      columns = ['awCap_factor', 'wiltingp_factor',
                                 'alpha_claycoef', 'alpha_sandcoef', 'alpha_siltcoef',
                                 'betaHBV_claycoef', 'betaHBV_sandcoef', 'betaHBV_siltcoef'])
    df['iparam'] = df.index + 50_000
    df.to_csv(f'{project_data_path}/WBM/precalibration/centralUS/params_extra_low.csv', index=False)

In [5]:
##########################################
### Uncertain parameters: append range ###
##########################################
n_sample = 5_000

if True:
    ###  Define bounds
    # awCap factor range ~ 2 - 5
    awCap_factor_lower, awCap_factor_upper = 2, 5

    # wiltingp factor range ~ 2 - 5
    wiltingp_factor_lower, wiltingp_factor_upper = 2, 5

    # alpha range ~ 1 - 10
    alpha_claycoef_lower, alpha_claycoef_upper = 0.0, 10.
    alpha_sandcoef_lower, alpha_sandcoef_upper = 0.0, 10.
    alpha_siltcoef_lower, alpha_siltcoef_upper = 0.0, 10.

    # betaHBV range ~ 1 - 10
    betaHBV_claycoef_lower, betaHBV_claycoef_upper = 0.0, 10.
    betaHBV_sandcoef_lower, betaHBV_sandcoef_upper = 0.0, 10.
    betaHBV_siltcoef_lower, betaHBV_siltcoef_upper = 0.0, 10.

    # GS_start_lower, GS_start_upper = 50, 200
    # GS_length_lower, GS_length_upper = 100, 200
    # rootDepth_oGS_lower, rootDepth_oGS_upper = 100, 1200
    # rootDepth_GS_factor_lower, rootDepth_GS_factor_upper = 2., 10.
    # rootDepth_all_lower, rootDepth_all_upper = 100, 2500

    # LHC sampling
    sampler = qmc.LatinHypercube(d=8)
    sample = sampler.random(n=n_sample)

    # l_bounds = [fieldCap_claycoef_lower, fieldCap_sandcoef_lower, fieldCap_siltcoef_lower,
    #             wiltingp_claycoef_lower, wiltingp_sandcoef_lower, wiltingp_siltcoef_lower,
    #             alpha_claycoef_lower, alpha_sandcoef_lower, alpha_siltcoef_lower,
    #             betaHBV_claycoef_lower, betaHBV_sandcoef_lower, betaHBV_siltcoef_lower]
    
    # u_bounds = [fieldCap_claycoef_upper, fieldCap_sandcoef_upper, fieldCap_siltcoef_upper,
    #             wiltingp_claycoef_upper, wiltingp_sandcoef_upper, wiltingp_siltcoef_upper,
    #             alpha_claycoef_upper, alpha_sandcoef_upper, alpha_siltcoef_upper,
    #             betaHBV_claycoef_upper, betaHBV_sandcoef_upper, betaHBV_siltcoef_upper]

    l_bounds = [awCap_factor_lower,
                wiltingp_factor_lower,
                alpha_claycoef_lower, alpha_sandcoef_lower, alpha_siltcoef_lower,
                betaHBV_claycoef_lower, betaHBV_sandcoef_lower, betaHBV_siltcoef_lower]
    
    u_bounds = [awCap_factor_upper,
                wiltingp_factor_upper,
                alpha_claycoef_upper, alpha_sandcoef_upper, alpha_siltcoef_upper,
                betaHBV_claycoef_upper, betaHBV_sandcoef_upper, betaHBV_siltcoef_upper]

    sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

    # Store as csv
    df = pd.DataFrame(data = sample_scaled,
                      columns = ['awCap_factor', 'wiltingp_factor',
                                 'alpha_claycoef', 'alpha_sandcoef', 'alpha_siltcoef',
                                 'betaHBV_claycoef', 'betaHBV_sandcoef', 'betaHBV_siltcoef'])
    df['iparam'] = df.index + 55_000
    df.to_csv(f'{project_data_path}/WBM/precalibration/centralUS/params_extra_high.csv', index=False)

In [15]:
# Merge
df_main = pd.read_csv(f'{project_data_path}/WBM/precalibration/centralUS/params.csv')
df_low = pd.read_csv(f'{project_data_path}/WBM/precalibration/centralUS/params_extra_low.csv')
df_high = pd.read_csv(f'{project_data_path}/WBM/precalibration/centralUS/params_extra_high.csv')

df = pd.concat([df_main, df_low, df_high])
df.to_csv(f'{project_data_path}/WBM/precalibration/centralUS/params_extra.csv', index=False)

In [12]:
len(df)

60000

In [11]:
len(df['iparam'].unique())

60000

In [8]:
############################
### Uncertain parameters ###
############################
n_sample = 50_000

if not os.path.isfile(f'{project_data_path}/WBM/precalibration/centralUS/params.csv'):
    ###  Define bounds
    # # fieldCap range ~ 0 - 0.75 (in m3/m3 -> multiply by 1000 to get mm/m)
    # fieldCap_claycoef_lower, fieldCap_claycoef_upper = 0.001, 0.75
    # fieldCap_sandcoef_lower, fieldCap_sandcoef_upper = 0.001, 0.75
    # fieldCap_siltcoef_lower, fieldCap_siltcoef_upper = 0.001, 0.75

    # # wilingp range ~ 0 - 1 (as factor of field capacity)
    # wiltingp_claycoef_lower, wiltingp_claycoef_upper = 0.001, 1.
    # wiltingp_sandcoef_lower, wiltingp_sandcoef_upper = 0.001, 1.
    # wiltingp_siltcoef_lower, wiltingp_siltcoef_upper = 0.001, 1.

    # awCap factor range ~ 0.5 - 2
    awCap_factor_lower, awCap_factor_upper = 0.5, 2

    # wiltingp factor range ~ 0.5 - 2
    wiltingp_factor_lower, wiltingp_factor_upper = 0.5, 2

    # alpha range ~ 1 - 10
    alpha_claycoef_lower, alpha_claycoef_upper = 0.0, 10.
    alpha_sandcoef_lower, alpha_sandcoef_upper = 0.0, 10.
    alpha_siltcoef_lower, alpha_siltcoef_upper = 0.0, 10.

    # betaHBV range ~ 1 - 10
    betaHBV_claycoef_lower, betaHBV_claycoef_upper = 0.0, 10.
    betaHBV_sandcoef_lower, betaHBV_sandcoef_upper = 0.0, 10.
    betaHBV_siltcoef_lower, betaHBV_siltcoef_upper = 0.0, 10.

    # GS_start_lower, GS_start_upper = 50, 200
    # GS_length_lower, GS_length_upper = 100, 200
    # rootDepth_oGS_lower, rootDepth_oGS_upper = 100, 1200
    # rootDepth_GS_factor_lower, rootDepth_GS_factor_upper = 2., 10.
    # rootDepth_all_lower, rootDepth_all_upper = 100, 2500

    # LHC sampling
    sampler = qmc.LatinHypercube(d=8)
    sample = sampler.random(n=n_sample)

    # l_bounds = [fieldCap_claycoef_lower, fieldCap_sandcoef_lower, fieldCap_siltcoef_lower,
    #             wiltingp_claycoef_lower, wiltingp_sandcoef_lower, wiltingp_siltcoef_lower,
    #             alpha_claycoef_lower, alpha_sandcoef_lower, alpha_siltcoef_lower,
    #             betaHBV_claycoef_lower, betaHBV_sandcoef_lower, betaHBV_siltcoef_lower]
    
    # u_bounds = [fieldCap_claycoef_upper, fieldCap_sandcoef_upper, fieldCap_siltcoef_upper,
    #             wiltingp_claycoef_upper, wiltingp_sandcoef_upper, wiltingp_siltcoef_upper,
    #             alpha_claycoef_upper, alpha_sandcoef_upper, alpha_siltcoef_upper,
    #             betaHBV_claycoef_upper, betaHBV_sandcoef_upper, betaHBV_siltcoef_upper]

    l_bounds = [awCap_factor_lower,
                wiltingp_factor_lower,
                alpha_claycoef_lower, alpha_sandcoef_lower, alpha_siltcoef_lower,
                betaHBV_claycoef_lower, betaHBV_sandcoef_lower, betaHBV_siltcoef_lower]
    
    u_bounds = [awCap_factor_upper,
                wiltingp_factor_upper,
                alpha_claycoef_upper, alpha_sandcoef_upper, alpha_siltcoef_upper,
                betaHBV_claycoef_upper, betaHBV_sandcoef_upper, betaHBV_siltcoef_upper]

    sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

    # Store as csv
    df = pd.DataFrame(data = sample_scaled,
                      columns = ['awCap_factor', 'wiltingp_factor',
                                 'alpha_claycoef', 'alpha_sandcoef', 'alpha_siltcoef',
                                 'betaHBV_claycoef', 'betaHBV_sandcoef', 'betaHBV_siltcoef'])
    df['iparam'] = df.index
    df.to_csv(f'{project_data_path}/WBM/precalibration/centralUS/params.csv', index=False)

In [5]:
# import seaborn as sns
# sns.pairplot(df.iloc[::100, :-1])

In [6]:
########################################################
########## SMAP and NLDAS data for validation ##########
########################################################
########################## SMAP
if not os.path.isfile(f'{project_data_path}/WBM/precalibration/centralUS/SMAP/SMAP_validation.nc'):
    # Read all
    files = glob(f'{smap_path}/processed_nldas_grid/SMAP_L4_SM_gph_all_nldas_*.nc')
    ds_smap = xr.concat([_subset_centralUS(xr.open_dataset(file)['sm_rootzone']) for file in files], dim='time')
    
    # Merge and store (and change units to kg/m3)
    ds_out = xr.Dataset({'soilMoist':1000*ds_smap})
    ds_out.attrs['units'] = 'kg/m3'
    ds_out.to_netcdf(f'{project_data_path}/WBM/precalibration/centralUS/SMAP_validation.nc')

    # Also store numpy array for quicker evaluations
    npy_out = np.transpose(ds_out['soilMoist'].to_numpy(), (2,1,0))
    np.save(f'{project_data_path}/WBM/precalibration/centralUS/SMAP_validation.npy', npy_out)

######################### NLDAS
nldas_dict = {'VIC':'SOILM0_100cm', 'NOAH':'SOILM', 'MOSAIC':'SOILM'}

for model, var_id in nldas_dict.items():
    if not os.path.isfile(f'{project_data_path}/WBM/precalibration/centralUS/{model}/{model}_validation.nc'):
        # Read all
        files = glob(f'{nldas_path}/{model}/daily/*.nc')
        ds_nldas = xr.concat([_subset_centralUS(xr.open_dataset(file)[var_id]) for file in files], dim='time')
    
        # Select correct depth
        if model in ['MOSAIC', 'NOAH']:
            ds_nldas = ds_nldas.isel(depth=1)
        else:
            ds_nldas = ds_nldas.isel(depth=0)
        
        # Merge and store
        ds_out = xr.Dataset({'soilMoist':ds_nldas})
        ds_out.attrs['units'] = 'kg/m3'
        ds_out.to_netcdf(f'{project_data_path}/WBM/precalibration/centralUS/{model}_validation.nc')

        # Also store numpy array for quicker evaluations
        npy_out = np.transpose(ds_out['soilMoist'].to_numpy(), (2,1,0))
        np.save(f'{project_data_path}/WBM/precalibration/centralUS/{model}_validation.npy', npy_out)

In [7]:
#############################################
### Climate forcing & geoophysical inputs ###
#############################################
for obs in ['MOSAIC', 'NOAH', 'VIC', 'SMAP']:
    if not os.path.isfile(f'{project_data_path}/WBM/precalibration/centralUS/{obs}/inputs.npz'):
        # Climate drivers
        if obs == "SMAP":
            files = glob(f'{smap_path}/processed_nldas_grid/SMAP_L4_SM_gph_all_nldas_*.nc')
            ds_forcing = xr.concat([_subset_centralUS(xr.open_dataset(file)) for file in files], dim='time')
        else:
            files = glob(f'{nldas_path}/forcing/daily/NLDAS_FORA0125_H.A*.nc')
            ds_forcing = _subset_centralUS(xr.concat([xr.open_dataset(file) for file in files], dim='time'))

        # Geophysical inputs
        ds_lai = _subset_centralUS(xr.open_dataset(f'{project_data_path}/WBM/geo_inputs/LAI_clima_NLDASgrid.nc'))
        if obs == 'VIC': # VIC does not provide awCap, wiltingp, so use NOAH
            obs_soilp = 'NOAH'
        else:
            obs_soilp = obs
        ds_awCap = _subset_centralUS(xr.open_dataset(f'{project_data_path}/WBM/geo_inputs/{obs_soilp}_awCap.nc'))
        ds_wiltingp = _subset_centralUS(xr.open_dataset(f'{project_data_path}/WBM/geo_inputs/{obs_soilp}_wiltingp.nc'))
    
        ds_clayfrac = _subset_centralUS(xr.open_dataset(f'{project_data_path}/WBM/geo_inputs/clayfrac_NLDASgrid.nc'))
        ds_sandfrac = _subset_centralUS(xr.open_dataset(f'{project_data_path}/WBM/geo_inputs/sandfrac_NLDASgrid.nc'))
        ds_siltfrac = _subset_centralUS(xr.open_dataset(f'{project_data_path}/WBM/geo_inputs/siltfrac_NLDASgrid.nc'))
    
        # Initial conditions
        ds_init = _subset_centralUS(xr.open_dataset(f'{project_data_path}/WBM/precalibration/centralUS/{obs}/{obs}_validation.nc')).isel(time=0)
    
        # Numpy arrays in correct order
        lats = ds_lai.lat.to_numpy()
        doy = ds_forcing.time.dt.dayofyear.to_numpy() - 1
    
        if obs == "SMAP":
            tas = np.transpose(ds_forcing['temp_lowatmmodlay'].to_numpy() - 273.15, (2,1,0))
            prcp = np.transpose(ds_forcing['precipitation_total_surface_flux'].to_numpy() * 86400, (2,1,0))
        else:
            tas = np.transpose(ds_forcing['TMP'].to_numpy() - 273.15, (2,1,0))
            prcp = np.transpose(ds_forcing['APCP'].to_numpy(), (2,1,0))
    
        lai = np.transpose(ds_lai['LAI'].to_numpy(), (2,1,0))
        awCap = np.transpose(ds_awCap['awCap'].to_numpy())
        wiltingp = np.transpose(ds_wiltingp['wiltingp'].to_numpy())
    
        clayfrac = np.transpose(ds_clayfrac['clayfrac'].to_numpy() / 100) # percentage -> fraction
        sandfrac = np.transpose(ds_sandfrac['sandfrac'].to_numpy() / 100) # percentage -> fraction
        siltfrac = np.transpose(ds_siltfrac['siltfrac'].to_numpy() / 100) # percentage -> fraction
    
        soilMoist_init = np.transpose(ds_init['soilMoist'].to_numpy())
    
        # Store numpy for easy access
        np.savez(f'{project_data_path}/WBM/precalibration/centralUS/{obs}/inputs.npz',
                 tas=tas, prcp=prcp,
                 lai=lai, awCap=awCap, wiltingp=wiltingp,
                 lats=lats, doy=doy,
                 clayfrac=clayfrac, sandfrac=sandfrac, siltfrac=siltfrac,
                 soilMoist_init=soilMoist_init)

## Run ensemble

In [8]:
# Import model code
from water_balance import simulate_water_balance

In [9]:
def run_WBMpy(forcing,
              alpha_claycoef,
              alpha_sandcoef,
              alpha_siltcoef,
              betaHBV_claycoef,
              betaHBV_sandcoef,
              betaHBV_siltcoef,
              Ts,
              Tm,
              Kc,
              awCap_factor,
              # fieldCap_claycoef,
              # fieldCap_sandcoef,
              # fieldCap_siltcoef,
              wiltingp_factor,
              # wiltingp_claycoef,
              # wiltingp_sandcoef,
              # wiltingp_siltcoef,
              Wi_init,
              Sp_init,
              GS_start,
              GS_length,
              rootDepth_oGS,
              rootDepth_GS_factor,
              iparam,
              store_path,
              simulate_water_balance):
    
    # Read and extract inputs
    npz = np.load(forcing)
    tas_in = npz['tas']
    prcp_in = npz['prcp']
    lai_in = npz['lai']
    awCap = npz['awCap']
    wiltingp = npz['wiltingp']
    lats_in = npz['lats']
    doy_in = npz['doy']
    clayfrac = npz['clayfrac']
    sandfrac = npz['sandfrac']
    siltfrac = npz['siltfrac']
    soilMoist_init = npz['soilMoist_init']
    
    # Array shapes
    nlon, nlat = soilMoist_init.shape
    grid = np.ones((nlon, nlat))

    # Alpha and beta have lower bound of 1
    alpha_in = 1. + (alpha_claycoef * clayfrac) + (alpha_sandcoef * sandfrac) + (alpha_siltcoef * siltfrac)
    betaHBV_in = 1. + (betaHBV_claycoef * clayfrac) + (betaHBV_sandcoef * sandfrac) + (betaHBV_siltcoef * siltfrac)

    # # field capacity in units of mm/m
    # fieldCap = 1000 * ((fieldCap_claycoef * clayfrac) + (fieldCap_sandcoef * sandfrac) + (fieldCap_siltcoef * siltfrac))
    # # wilting point is fraction of fieldCap
    # wiltingp_in = fieldCap * ((wiltingp_claycoef * clayfrac) + (wiltingp_sandcoef * sandfrac) + (wiltingp_siltcoef * siltfrac))
    # # construct awCap such that this relation holds
    # awCap_in = fieldCap - wiltingp_in

    # awCap
    awCap_in = awCap_factor * awCap

    # wiltingp
    wiltingp_in = wiltingp_factor * wiltingp

    # WBM models the active profile so let starting soil moisture reflect this
    Ws_init_in = soilMoist_init - wiltingp_in
    Ws_init_in = np.where(Ws_init_in >= 0., Ws_init_in, 0.)

    # Other inputs
    Kc_in = Kc * np.ones(lai_in.shape)
    Wi_init_in = Wi_init * grid
    Sp_init_in = Sp_init * grid
    GS_start_in = GS_start * grid
    GS_length_in = GS_length * grid
    rootDepth_oGS_in = rootDepth_oGS * grid
    rootDepth_GS_factor_in = rootDepth_GS_factor * grid

    # Run it
    out = simulate_water_balance(
        Ws_init = Ws_init_in,
        Wi_init = Wi_init_in,
        Sp_init = Sp_init_in,
        P = prcp_in,
        T = tas_in,
        Ts = Ts,
        Tm = Tm,
        rootDepth_oGS = rootDepth_oGS_in,
        rootDepth_GS_factor = rootDepth_GS_factor_in,
        awCap = awCap_in,
        wilting_point = wiltingp_in,
        GS_start = GS_start_in,
        GS_length = GS_length_in,
        lai = lai_in,
        Kc = Kc_in,
        alpha = alpha_in,
        betaHBV = betaHBV_in,
        phi = lats_in,
        doy = doy_in
    )

    # Store
    np.save(f'{store_path}/{str(iparam)}.npy', out)

In [10]:
# Run ensemble
df_param = pd.read_csv(f'{project_data_path}/WBM/precalibration/centralUS/params_extra.csv')

# Constants
Ts = -1.             # Snowfall threshold
Tm = 1.              # Snowmelt threshold
Wi_init = 0.         # Initial canopy water storage
Sp_init = 0.         # Initial snowpack

# Neglect influence of growing season for now
GS_start = 0 # meaningless
GS_length = 0 # meaningless
rootDepth_oGS = 1000
rootDepth_GS_factor = 1
Kc = 1

# Dask delayed
delayed = []
for obs in ['MOSAIC', 'NOAH', 'VIC', 'SMAP']:
    for iparam in range(50_000, 60_000):
        # Check if done
        if not os.path.isfile(f'{project_data_path}/WBM/precalibration/centralUS/{obs}/out/{str(iparam)}.npy'):
        
            # Get uncertain parameters
            alpha_claycoef = df_param[df_param.iparam == iparam]['alpha_claycoef'].values[0]
            alpha_sandcoef = df_param[df_param.iparam == iparam]['alpha_sandcoef'].values[0]
            alpha_siltcoef = df_param[df_param.iparam == iparam]['alpha_siltcoef'].values[0]
            
            betaHBV_claycoef = df_param[df_param.iparam == iparam]['betaHBV_claycoef'].values[0]
            betaHBV_sandcoef = df_param[df_param.iparam == iparam]['betaHBV_sandcoef'].values[0]
            betaHBV_siltcoef = df_param[df_param.iparam == iparam]['betaHBV_siltcoef'].values[0]

            awCap_factor = df_param[df_param.iparam == iparam]['awCap_factor'].values[0]
            # fieldCap_claycoef = df_param[df_param.iparam == iparam]['fieldCap_claycoef'].values[0]
            # fieldCap_sandcoef = df_param[df_param.iparam == iparam]['fieldCap_sandcoef'].values[0]
            # fieldCap_siltcoef = df_param[df_param.iparam == iparam]['fieldCap_siltcoef'].values[0]

            wiltingp_factor = df_param[df_param.iparam == iparam]['wiltingp_factor'].values[0]
            # wiltingp_claycoef = df_param[df_param.iparam == iparam]['wiltingp_claycoef'].values[0]
            # wiltingp_sandcoef = df_param[df_param.iparam == iparam]['wiltingp_sandcoef'].values[0]
            # wiltingp_siltcoef = df_param[df_param.iparam == iparam]['wiltingp_siltcoef'].values[0]

            # Delayed calculation
            tmp = dask.delayed(run_WBMpy)(
                forcing = f'{project_data_path}/WBM/precalibration/centralUS/{obs}/inputs.npz',
                alpha_claycoef = alpha_claycoef,
                alpha_sandcoef = alpha_sandcoef,
                alpha_siltcoef = alpha_siltcoef,
                betaHBV_claycoef = betaHBV_claycoef,
                betaHBV_sandcoef = betaHBV_sandcoef,
                betaHBV_siltcoef = betaHBV_siltcoef,
                awCap_factor = awCap_factor,
                wiltingp_factor = wiltingp_factor,
                # fieldCap_claycoef = fieldCap_claycoef,
                # fieldCap_sandcoef = fieldCap_sandcoef,
                # fieldCap_siltcoef = fieldCap_siltcoef,
                # wiltingp_claycoef = wiltingp_claycoef,
                # wiltingp_sandcoef = wiltingp_sandcoef,
                # wiltingp_siltcoef = wiltingp_siltcoef,
                Ts = Ts,
                Tm = Tm,
                Kc = Kc,
                Wi_init = Wi_init,
                Sp_init = Sp_init,
                GS_start = GS_start,
                GS_length = GS_length,
                rootDepth_oGS = rootDepth_oGS,
                rootDepth_GS_factor = rootDepth_GS_factor,
                iparam = iparam,
                store_path = f'{project_data_path}/WBM/precalibration/centralUS/{obs}/out',
                simulate_water_balance = simulate_water_balance
            )

            delayed.append(tmp)

In [11]:
len(delayed)

40000

In [12]:
%%time
# Compute
results = dask.compute(*delayed)

/storage/home/dcl5300/miniforge3/envs/climate-stack-mamba-2023-11-01/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 31.66 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 44min 27s, sys: 2min 19s, total: 46min 47s
Wall time: 7h 27min 15s


## Analysis

### Calculate metrics

In [6]:
# Get Pearson correlation and RMSE
def calculate_rmse(obs, iparam, tmin, project_data_path, log_path):
    if os.path.isfile(f'{project_data_path}/WBM/precalibration/centralUS/{obs}/out/{str(iparam)}.npy'):
        # Read simulation output
        sim_tmp = np.load(f'{project_data_path}/WBM/precalibration/centralUS/{obs}/out/{str(iparam)}.npy')

        # Read obs
        ds_obs = np.load(f'{project_data_path}/WBM/precalibration/centralUS/{obs}/{obs}_validation.npy')

        # RMSE
        def calculate_rmse_global(predictions, targets):
            return np.sqrt(np.nanmean((predictions-targets)**2))

        def calculate_rmse_gpa(predictions, targets):
            # Assumes time axis is -1
            return np.nanmean(np.sqrt(np.nanmean(((predictions-targets)**2), axis=-1)))

        def calculate_rmse_ngpa(predictions, targets):
            # Assumes time axis is -1
            rmse = np.sqrt(np.nanmean(((predictions-targets)**2), axis=-1))
            mean = np.nanmean(targets, axis=-1)
            return np.nanmean(rmse/mean)

        # Skip first tmin timesteps for spinup
        rmse_gpa = calculate_rmse_gpa(sim_tmp[:,:,tmin:], ds_obs[:,:,tmin:])
        # rmse_ngpa = calculate_rmse_ngpa(sim_tmp[:,:,tmin:], ds_obs[:,:,tmin:])

        # Merge all
        df_out = pd.DataFrame(data = {
            'rmse_gpa': [rmse_gpa],
            # 'rmse_ngpa': [rmse_ngpa],
            'iparam': [iparam],
        })
        
        # Return spatial average
        return df_out
    else:
        return None
            
    # except Exception as e:
    #      with open(f'{log_path}/rmse_{obs}_{str(iparam)}.txt', 'w') as f:
    #         f.write(str(e))
    #         return None

In [7]:
%%time
# Skip first tmin days
tmin = 0

for obs in ['NOAH', 'MOSAIC', 'VIC', 'SMAP']:
    # Delayed
    delayed = []

    for iparam in range(60_000):
        delayed.append(dask.delayed(calculate_rmse)(obs = obs,
                                                    iparam = iparam,
                                                    tmin = tmin,
                                                    project_data_path = project_data_path,
                                                    log_path = log_path))

    # Compute
    out = dask.compute(*delayed)

    # Store (append if any new results)
    out_path = f'{project_data_path}/WBM/precalibration/centralUS/{obs}/soilMoist_rmse_skip{str(tmin)}.csv'
    if os.path.isfile(out_path):
        # Read existing
        df_res = pd.read_csv(out_path)
        # Append new
        df_out = pd.concat(out, ignore_index=True)
        df_res = pd.concat([df_res, df_out], ignore_index=True)
        # Drop duplicates
        df_res = df_res.drop_duplicates(subset='iparam')
        # Store
        df_res.to_csv(out_path, index=False)
    else:
        df_out = pd.concat(out, ignore_index=True)
        df_out.to_csv(out_path, index=False)

/storage/home/dcl5300/miniforge3/envs/climate-stack-mamba-2023-11-01/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 10.88 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/storage/home/dcl5300/miniforge3/envs/climate-stack-mamba-2023-11-01/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 10.88 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/storage/home/dcl5300/miniforge3/envs/climate-stack-mamba-2023-11-01/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 10.88 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/storage/home/dcl5300/miniforge3/envs/climate-stack-mamba-2023-11-01/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending larg

CPU times: user 13min 15s, sys: 35.8 s, total: 13min 51s
Wall time: 1h 21min 38s


### OLD

In [ ]:
######################## OLD  
# Use weekly anomalies to validate across different obs
# ds_nldas = ds_nldas.assign_coords(week=ds_nldas.time.dt.strftime("%W"))
# ds_nldas_anom = (ds_nldas.groupby('week') - ds_nldas.groupby("week").mean("time"))

# np.save(f'{project_data_path}/WBM/precalibration/centralUS/{model}_obs_anoms_weekly.npy',
#         np.transpose(ds_nldas_anom[var_id].to_numpy(), (2,1,0)))

In [3]:
# Get Pearson correlation and RMSE
def calculate_error_metrics(project_data_path, iparam):
    try:
        # Read simulation output
        sim_tmp = np.load(f'{project_data_path}/WBM/precalibration/centralUS/out/{str(iparam)}_Ws.npy')

        # Read SMAP for coords
        smap_obs = xr.open_dataset(f'{project_data_path}/WBM/precalibration/centralUS/SMAP_validation.nc').drop(['soilMoist_anom', 'week'])

        # Construct xr 
        ds_sim = xr.Dataset(
            data_vars=dict(soilMoist=(["time", "lat", "lon"], np.transpose(sim_tmp, (2,1,0)))),
            coords=dict(
                lon=smap_obs.lon,
                lat=smap_obs.lat,
                time=smap_obs.time)
        )

        # Get anomaly from weekly climatology
        ds_sim['soilMoist_clima'] = ds_sim['soilMoist'].groupby(ds_sim.time.dt.isocalendar().week).mean("time")
        ds_sim['soilMoist_anom'] = (ds_sim['soilMoist'].groupby(ds_sim.time.dt.isocalendar().week) - ds_sim['soilMoist_clima'])
        
        smap_obs['soilMoist_clima'] = smap_obs['soilMoist'].groupby(smap_obs.time.dt.isocalendar().week).mean("time")
        smap_obs['soilMoist_anom'] = (smap_obs['soilMoist'].groupby(smap_obs.time.dt.isocalendar().week) - smap_obs['soilMoist_clima'])
        
        # Read other observations
        def calculate_weekly_clima(ds):
            ds_tmp = ds.copy().drop(['soilMoist_anom', 'week'])
            ds_tmp['soilMoist_clima'] = ds_tmp['soilMoist'].groupby(ds_tmp.time.dt.isocalendar().week).mean("time")
            ds_tmp['soilMoist_anom'] = (ds_tmp['soilMoist'].groupby(ds_tmp.time.dt.isocalendar().week) - ds_tmp['soilMoist_clima'])
            return ds_tmp
            
        noah_obs = calculate_weekly_clima(xr.open_dataset(f'{project_data_path}/WBM/precalibration/centralUS/NOAH_validation.nc'))
        vic_obs = calculate_weekly_clima(xr.open_dataset(f'{project_data_path}/WBM/precalibration/centralUS/VIC_validation.nc'))
        mosaic_obs = calculate_weekly_clima(xr.open_dataset(f'{project_data_path}/WBM/precalibration/centralUS/MOSAIC_validation.nc'))

        # Person correlation
        # smap_corr = xr.corr(ds_sim['soilMoist'].isel(time=slice(100,3000)), smap_obs['soilMoist'].isel(time=slice(100,3000)), dim='time')
        # vic_corr = xr.corr(ds_sim['soilMoist'].isel(time=slice(100,3000)), vic_obs['soilMoist'].isel(time=slice(100,3000)), dim='time')
        # mosaic_corr = xr.corr(ds_sim['soilMoist'].isel(time=slice(100,3000)), mosaic_obs['soilMoist'].isel(time=slice(100,3000)), dim='time')
        # noah_corr = xr.corr(ds_sim['soilMoist'].isel(time=slice(100,3000)), noah_obs['soilMoist'].isel(time=slice(100,3000)), dim='time')

        # smap_corr = xr.corr(ds_sim['soilMoist_clima'], smap_obs['soilMoist_clima'], dim='time')
        # vic_corr = xr.corr(ds_sim['soilMoist_clima'], vic_obs['soilMoist_clima'], dim='time')
        # mosaic_corr = xr.corr(ds_sim['soilMoist_clima'], mosaic_obs['soilMoist_clima'], dim='time')
        # noah_corr = xr.corr(ds_sim['soilMoist_clima'], noah_obs['soilMoist_clima'], dim='time')
        
        smap_corr_anom = xr.corr(ds_sim['soilMoist_anom'].isel(time=slice(100,3000)), smap_obs['soilMoist_anom'].isel(time=slice(100,3000)), dim='time')
        vic_corr_anom = xr.corr(ds_sim['soilMoist_anom'].isel(time=slice(100,3000)), vic_obs['soilMoist_anom'].isel(time=slice(100,3000)), dim='time')
        mosaic_corr_anom = xr.corr(ds_sim['soilMoist_anom'].isel(time=slice(100,3000)), mosaic_obs['soilMoist_anom'].isel(time=slice(100,3000)), dim='time')
        noah_corr_anom = xr.corr(ds_sim['soilMoist_anom'].isel(time=slice(100,3000)), noah_obs['soilMoist_anom'].isel(time=slice(100,3000)), dim='time')

        # RMSE
        def _calculate_rmse(ds1, ds2):
            return np.sqrt(((ds1 - ds2)**2).mean(dim='week'))

        # smap_rmse = _calculate_rmse(ds_sim['soilMoist'].isel(time=slice(100,3000)), smap_obs['soilMoist'].isel(time=slice(100,3000)))
        # vic_rmse = _calculate_rmse(ds_sim['soilMoist'].isel(time=slice(100,3000)), vic_obs['soilMoist'].isel(time=slice(100,3000)))
        # mosaic_rmse = _calculate_rmse(ds_sim['soilMoist'].isel(time=slice(100,3000)), mosaic_obs['soilMoist'].isel(time=slice(100,3000)))
        # noah_rmse = _calculate_rmse(ds_sim['soilMoist'].isel(time=slice(100,3000)), noah_obs['soilMoist'].isel(time=slice(100,3000)))

        smap_rmse = _calculate_rmse(ds_sim['soilMoist_clima'], smap_obs['soilMoist_clima'])
        vic_rmse = _calculate_rmse(ds_sim['soilMoist_clima'], vic_obs['soilMoist_clima'])
        mosaic_rmse = _calculate_rmse(ds_sim['soilMoist_clima'], mosaic_obs['soilMoist_clima'])
        noah_rmse = _calculate_rmse(ds_sim['soilMoist_clima'], noah_obs['soilMoist_clima'])
        
        # smap_rmse_anom = _calculate_rmse(ds_sim['soilMoist_anom'].isel(time=slice(100,3000)), smap_obs['soilMoist_anom'].isel(time=slice(100,3000)))
        # vic_rmse_anom = _calculate_rmse(ds_sim['soilMoist_anom'].isel(time=slice(100,3000)), vic_obs['soilMoist_anom'].isel(time=slice(100,3000)))
        # mosaic_rmse_anom = _calculate_rmse(ds_sim['soilMoist_anom'].isel(time=slice(100,3000)), mosaic_obs['soilMoist_anom'].isel(time=slice(100,3000)))
        # noah_rmse_anom = _calculate_rmse(ds_sim['soilMoist_anom'].isel(time=slice(100,3000)), noah_obs['soilMoist_anom'].isel(time=slice(100,3000)))

        # Merge all
        ds_out = xr.Dataset({
            # 'smap_pearsonr': smap_corr,
            'smap_pearsonr_anom': smap_corr_anom,
            # 'vic_pearsonr': vic_corr,
            'vic_pearsonr_anom': vic_corr_anom,
            # 'noah_pearsonr': noah_corr,
            'noah_pearsonr_anom': noah_corr_anom,
            # 'mosaic_pearsonr': mosaic_corr,
            'mosaic_pearsonr_anom': mosaic_corr_anom,
            'smap_rmse_clima': smap_rmse,
            # 'smap_rmse_anom': smap_rmse_anom,
            'vic_rmse_clima': vic_rmse,
            # 'vic_rmse_anom': vic_rmse_anom,
            'noah_rmse_clima': noah_rmse,
            # 'noah_rmse_anom': noah_rmse_anom,
            'mosaic_rmse_clima': mosaic_rmse,
            # 'mosaic_rmse_anom': mosaic_rmse_anom,
        })
        
        # Return spatial average
        return ds_out.mean(dim=['lat','lon']).assign_coords(iparam=[iparam]).to_dataframe()
            
    except Exception as e:
         with open(f'/storage/work/dcl5300/current_projects/wbm_soilM_crop_uc_lafferty-etal-2024-tbd/code/logs/{str(iparam)}.txt', 'w') as f:
            f.write(str(e))
            return None

In [7]:
# Delayed
delayed = []

for iparam in range(10_000):
    tmp = dask.delayed(calculate_error_metrics)(project_data_path = project_data_path,
                                                iparam = iparam)
    delayed.append(tmp)

In [ ]:
%%time
# Compute
out = dask.compute(*delayed)

# Store
pd.concat(out).to_csv(f'{project_data_path}/WBM/precalibration/centralUS/results_2step.csv')